# Practical 3: Tree methods

Computational Finance with Python

[Alet Roux](https://www.york.ac.uk/maths/staff/alet-roux/) ([Department
of Mathematics](https://maths.york.ac.uk), University of York)

The aim of this practical is to practice using Python in a tree model.

Click on the following to open this file in Google Colab:

<figure>
<a
href="https://colab.research.google.com/github/aletroux/comp-finance-python/blob/main/practicals/03_tree_methods_prac.ipynb"><img
src="https://colab.research.google.com/assets/colab-badge.svg"
alt="Open In Colab" /></a>
<figcaption>Open In Colab</figcaption>
</figure>

In the previous practical, we developed the following code for pricing a
European option in the Cox-Ross-Rubinstein model. Run the code cell
below so that you can use the functions defined in it.

In [1]:
import numpy as np

def Eoptionprice (S0, N, u, r, d, dt, payoff):
    """Price a European option in the Cox-Ross-Rubinstein model.
    Arguments:
    S0: initial price
    N: number of steps
    u: up jump, u > e^{r dt}
    r: continuously compounded interest rate
    d: down jump, d < e^{r dt}
    dt: size of time step
    payoff: payoff function
    Output:
    Price of call at time 0."""

    # Risk-neutral probability
    q = (math.exp(r*dt) - d)/(u - d)
    
    # This value will be used frequently.
    # Calculate it with math.exp rather than np.exp as it is faster on scalars.
    discount = math.exp(-r*dt)
        
    # Initialise two-dimensional array V.
    # Initial values equal to 0 (but could also have used empty).
    # V[n, k] = node k at time step n.
    # remember this is a matrix but only half of the positions are used
    V = np.zeros((N+1,N+1))
    
    # Step 1: Calculate V_N
    ks = np.arange(N+1)
    # ks[::-1] is just a reversed view of ks
    S = S0 * u**ks * d**ks[::-1]
    V[N] = np.vectorize(payoff, otypes=[np.float64])(S)
    
    # Step 2: Now iterate backwards
    for n in np.arange(N-1,-1,-1):
        # loop replaced by NumPy operations
        # notice selection of sub-arrays in the calculation
        V[n, :n+1] = discount*(q*V[n+1, 1:n+2] + (1-q)*V[n+1, :n+1])

    # Step 3
    return V[0, 0]

def putpayoff(S,K):
    """Put option payoff function. Arguments are
    S: Stock price
    K: Strike price"""
    return max(K-S,0)

def callpayoff(S,K):
    """Call option payoff function. Arguments:
    S: Stock price
    K: Strike price"""
    return max(S-K,0)


Assume the stock price follows the Black-Scholes model. A
Cox-Ross-Rubinstein model with $N$ steps can be built to approximate the
stock price process, with parameters are given by the following:
$$\left.\begin{aligned}
u_N &= \beta_N + \sqrt{\beta^2_N-\gamma}, \\
d_N &= \beta_N - \sqrt{\beta^2_N-\gamma},
\end{aligned}\right\}$$ where
$$\beta_N = \frac{1}{2}\left(\gamma e^{-r\Delta t}+ e^{(r+\sigma^2)\Delta t}\right).$$
Here $\gamma>0$ is a parameter that needs to be chosen to suit the
model, and $\Delta t=T/N$, where $T$ is the time horizon (such as option
maturity date) in the Black-Scholes model.

<span class="theorem-title">**Exercise 1**</span> Complete the function
in the following code cell. Consult the `math` documentation (The Python
Software Foundation (2024)) for any mathematical functions that you
might need.

In [2]:
import math

def CRRparameters (N, T, r, sigma, gamma = 1):
    """Calculates parameters dt, u and d of a CRR model approximating a Black-Scholes 
    model with N steps, where the Black-Scholes model has time horizon T and parameters
    r and sigma."""

    #insert code here.
    
    return dt, u, d

<span class="theorem-title">**Exercise 2**</span> Now use these
functions to perform a binomial tree approximation with $N=100$ to the
price of a call option with strike price $K=95$ and three month maturity
($T=0.25$), where the Black-Scholes parameters are $S_0=90$, $r=0.03$
and $\sigma=0.2$. You should choose
$$\gamma_N = e^{\frac{2}{N}\ln \frac{K}{S_0}}.$$

In [4]:
#Insert code here.

<span class="theorem-title">**Exercise 3**</span> The next step is to
study the order of convergence empirically. We use the same parameter
values as above, but vary $N$ (and hence
$\gamma_N, \Delta t, u_N, d_N, q_N$. Consider the sequence $N=2^n$ where
$n=1,2,3,\ldots$ (choose an appropriate upper bound for $n$—start
small!). Thus $N$ is doubled every time.

The following code cell contains a function that can be used to plot
sequences, and a function to calculate the price $C$ of a call option in
the Black-Scholes model. The latter uses SciPy (Virtanen et al. (2020));
documentation can be found at the [SciPy website](https://scipy.org/).

1.  Create a plot of the sequence $(C_N)$. What do you notice?

2.  Calculate the sequence
    $$ \frac{C_{4}-C}{C_{2}-C}, \frac{C_{8}-C}{C_{4}-C}, \frac{C_{16}-C}{C_{8}-C}, \ldots $$
    where $C$ is the call option price in the Black-Scholes model.
    Create a plot of this sequence. What do you notice?

3.  Is it reasonable to conclude that if the number of time steps is
    doubled, then the error should be approximately halved?

In [6]:
from scipy.stats import norm
import matplotlib.pyplot as plt

def BScallprice (S0, K, T, r, sigma):
    """Calculates the theoretical price at time 0 of a European call option with strike K and maturity 
    date T in the Black-Scholes model with parameters r and sigma."""

    diff = sigma*math.sqrt(T)
    dplus = (math.log(S0/K) + (r+0.5*sigma**2)*T)/diff
    dminus = dplus - diff
    price = S0 * norm.cdf(dplus) - math.exp(-r*T) * K * norm.cdf(dminus)

    return price

def plotN (n, CN, titletext = "insert title here"):
    """Plots values CN against integer n"""
    fig, ax = plt.subplots()
    ax.plot(n, CN, marker = 'o')
    ax.xaxis.grid(True)
    ax.yaxis.grid(True)
    ax.set(title=titletext,xlabel='$n$')
    plt.show(fig)

# Insert code here

<span class="theorem-title">**Exercise 4**</span> Now modify the code
above so that it provides one more output, namely the delta ($\Delta$).
Use the method of adding two steps to the tree, as covered in the
lecture slides.

Then analyse the convergence of the approximations to the Black-Scholes
delta (the formula for this can be found in the lecture notes) in a
similar way as in
<a href="#exr-convergence" class="quarto-xref">Exercise 3</a> above.

# References

The Python Software Foundation. 2024. “<span class="nocase">math</span>:
Mathematical Functions.”
<https://docs.python.org/3/library/decimal.html>.

Virtanen, Pauli, Ralf Gommers, Travis E. Oliphant, Matt Haberland, Tyler
Reddy, David Cournapeau, Evgeni Burovski, et al. 2020.
“<span class="nocase">SciPy 1.0: Fundamental Algorithms for Scientific
Computing in Python</span>.” *Nature Methods* 17: 261–72.
<https://doi.org/10.1038/s41592-019-0686-2>.